In [1]:
import os.path
import os
import numpy as np
import pandas as pd
import re

In [2]:
df_raw = pd.read_csv('./result2.csv', sep=';')

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1506 entries, 0 to 1505
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Цена за квадрат     1506 non-null   object 
 1   Тип дома            792 non-null    object 
 2   Кухня               955 non-null    object 
 3   Лифты               780 non-null    object 
 4   Название            1506 non-null   object 
 5   Строительная серия  193 non-null    object 
 6   Адрес               1506 non-null   object 
 7   Этаж                1506 non-null   object 
 8   Комнаты             1500 non-null   object 
 9   Отопление           517 non-null    object 
 10  Парковка            597 non-null    object 
 11  Общая               1506 non-null   object 
 12  Подъезды            536 non-null    float64
 13  Год постройки       635 non-null    float64
 14  Тип перекрытий      490 non-null    object 
 15  Метро               1460 non-null   object 
 16  Жилая 

Удалим все столбцы, в которых отсутсвует более 45% данных

In [4]:
sum(df_raw.isna().sum() / df_raw.shape[0] > 0.45)

14

In [5]:
empty_data = df_raw.isna().sum().sort_values()[-16:]

In [6]:
df_raw.drop(empty_data.index, axis=1, inplace=True)

Удалим столбец "название" так как для обучения модели он не даст полезной информации и "Комнаты" т.к туда попала информация о ремонте + много мусора про риелтора и прочее

In [7]:
df_raw.drop('Название', axis=1, inplace=True)

In [8]:
df_raw.drop('Комнаты', axis=1, inplace=True)

Разделим столбец Этаж на два: фактический этаж квартиры и высота всего здания

In [9]:
split_columns = df_raw['Этаж'].str.split(' ', expand=True)

In [10]:
df_raw['Этаж'] = split_columns[0]
df_raw['Этажность здания'] = split_columns[2]

In [11]:
df_raw['Этаж'] = df_raw['Этаж'].astype(int)

In [12]:
df_raw['Этажность здания'] = df_raw['Этажность здания'].astype(int)

Очистим столбцы от ненужных символов

In [13]:
split_columns = df_raw['Общая'].str.replace(',', '.').str.replace(' ', '').str.split('\xa0', expand=True)
df_raw['Общая'] = split_columns[0].astype(float)

In [14]:
df_raw['Цена за квадрат'] = df_raw['Цена за квадрат'].str.replace(' ', '').str.replace('\xa0', '').astype(float)

Разделим время до метро на кол-во минут и способ передвижения

In [15]:
split_subway = df_raw['Время до метро'].str.split(' ', n=4, expand=True)

In [16]:
df_raw['Время'] = split_subway[2]
df_raw['Способ передвижения'] = split_subway[4]

In [17]:
df_raw = df_raw.drop(['Время до метро'], axis=1)

Проверим, какие данные у нас получились во времени и заменим на более точные данные, плюс отфильтруем по тем, что более 30 мин

In [18]:
df_raw['Время'].unique()

array(['13', '5', '6', '14', '20', '7', '1', '2', '19', '9', '8', '3',
       '15', '26', '28', '12', '10', '4', '18', '23', '11', '17', '22',
       '16', nan, '32', '25', '29', '<1', '21', '24', '27', '2022', '54',
       '40', '50', '30'], dtype=object)

In [19]:
df_raw['Время'] = df_raw['Время'].str.replace('<1', '0').astype(float)

In [20]:
df_raw = df_raw[df_raw['Время'] < 30]

Удалим строки, в которых значения таргета неизвестно, если такие есть

In [21]:
df_raw.isna().sum().sort_values()

Цена за квадрат        0
Адрес                  0
Этаж                   0
Общая                  0
Метро                  0
Цена                   0
Этажность здания       0
Время                  0
Способ передвижения    0
dtype: int64

In [22]:
df_raw = df_raw[df_raw['Цена'].notna()]

Переведем наши адреса в координаты

In [23]:
from geopy.geocoders import GoogleV3

In [24]:
geolocator = GoogleV3(api_key='user') #стерла свой адрес, при использовании подключала аккаунт гугла

In [29]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.1)

In [30]:
geocode('Москва, метро Лианозово', timeout=10)

Location(Lianozovo, Moskva, Russia, 127253, (55.897353, 37.553153, 0.0))

In [31]:
df_raw['Город и метро'] = 'Москва, метро ' + df_raw['Метро']

In [32]:
df_raw['Город и метро'] = df_raw['Город и метро'].apply(geocode, timeout=10)
df_raw['Координаты метро'] = df_raw['Город и метро'].apply(lambda loc: tuple(loc.point) if loc else None)

In [34]:
df_raw['Адрес'] = df_raw['Адрес'].apply(geocode, timeout=10)

In [35]:
df_raw['Координаты дома'] = df_raw['Адрес'].apply(lambda loc: tuple(loc.point) if loc else None)

In [37]:
df_raw = df_raw[df_raw['Координаты дома'].notna()]

In [38]:
df_raw['Широта дома'] = df_raw['Координаты дома'].apply(lambda x: x[0])
df_raw['Долгота дома'] = df_raw['Координаты дома'].apply(lambda x: x[1])

In [39]:
df_raw['Широта метро'] = df_raw['Координаты метро'].apply(lambda x: x[0])
df_raw['Долгота метро'] = df_raw['Координаты метро'].apply(lambda x: x[1])

In [40]:
df_raw = df_raw.drop(['Координаты метро', 'Координаты дома', 'Город и метро'], axis=1)

In [41]:
df_raw = df_raw.drop(columns=['Адрес', 'Способ передвижения', 'Время'], axis=1)

Добавим расстояние от дома до центра города

In [42]:
from geopy import distance

lon_center, lat_center = 55.753595, 37.621031

In [43]:
df_raw['До центра'] = df_raw[['Широта дома', 'Долгота дома']].apply(
    lambda x: distance.distance((x[0],x[1]),(lon_center, lat_center)).km, axis = 1
)

In [44]:
df_raw['До метро'] = df_raw[['Широта дома', 'Долгота дома', 'Широта метро', 'Долгота метро']].apply(
    lambda x: distance.distance((x[0],x[1]),(x[2],x[3])).km, axis = 1
)

In [45]:
df_raw = df_raw[df_raw['До центра'] < 35]
df_raw.shape

(1356, 12)

In [46]:
import math

In [47]:
def azimute(long, lat):
    #pi - число pi, rad - радиус сферы (Земли)
    rad = 6372795
 
    #координаты двух точек
    llat1 = lat
    llong1 = long

    llat2 = lat_center
    llong2 = lon_center

    #в радианах
    lat1 = llat1*math.pi/180.
    lat2 = llat2*math.pi/180.
    long1 = llong1*math.pi/180.
    long2 = llong2*math.pi/180.

    #косинусы и синусы широт и разницы долгот
    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)

    #вычисления длины большого круга
    y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)
    dist = ad*rad

    #вычисление начального азимута
    x = (cl1*sl2) - (sl1*cl2*cdelta)
    y = sdelta*cl2
    z = math.degrees(math.atan(-y/x))

    if (x < 0):
        z = z+180.

    z2 = (z+180.) % 360. - 180.
    z2 = - math.radians(z2)
    anglerad2 = z2 - ((2*math.pi)*math.floor((z2/(2*math.pi))) )
    angledeg = (anglerad2*180.)/math.pi
    
    return round(angledeg, 3)

In [48]:
df_raw['Азимут'] = df_raw[['Широта дома', 'Долгота дома']].apply(
    lambda x: azimute(x[0],x[1]), axis = 1
)

In [49]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1356 entries, 0 to 1503
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Цена за квадрат   1356 non-null   float64
 1   Этаж              1356 non-null   int64  
 2   Общая             1356 non-null   float64
 3   Метро             1356 non-null   object 
 4   Цена              1356 non-null   float64
 5   Этажность здания  1356 non-null   int64  
 6   Широта дома       1356 non-null   float64
 7   Долгота дома      1356 non-null   float64
 8   Широта метро      1356 non-null   float64
 9   Долгота метро     1356 non-null   float64
 10  До центра         1356 non-null   float64
 11  До метро          1356 non-null   float64
 12  Азимут            1356 non-null   float64
dtypes: float64(10), int64(2), object(1)
memory usage: 148.3+ KB


In [52]:
df_raw.to_csv('clean_data.csv', index=False)

In [51]:
df_raw

,Цена за квадрат,Этаж,Общая,Метро,Цена,Этажность здания,Широта дома,Долгота дома,Широта метро,Долгота метро,До центра,До метро,Азимут
0,752522.0,9,99.52,Парк Победы,74890993.0,10,55.731342,37.514950,55.735885,37.517492,7.108388,0.530374,9.433
1,279208.0,4,60.60,Лианозово,16920000.0,16,55.894126,37.521438,55.897353,37.553153,16.845703,2.016353,311.844
2,2117647.0,5,139.90,Смоленская,296258823.0,5,55.748779,37.589725,55.747261,37.581945,2.037621,0.516970,6.947
3,548263.0,3,129.50,Минская,71000000.0,14,55.724524,37.496757,55.724791,37.496755,8.450356,0.029683,10.497
4,445920.0,6,52.70,Беломорская,23500000.0,20,55.869825,37.457761,55.864057,37.463808,16.499986,0.745486,330.591
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,250000.0,5,16.20,Андроновка,4050000.0,6,55.748092,37.731480,55.747354,37.738067,6.962497,0.421743,177.740
1499,442857.0,4,14.00,Проспект Вернадского,6200000.0,10,55.673548,37.503676,55.677108,37.505945,11.568649,0.421240,28.375
1500,547619.0,2,21.00,Авиамоторная,11500000.0,18,55.756947,37.700501,55.751077,37.717072,5.003589,1.228690,181.913
1502,300905.0,4,44.20,Беломорская,13300000.0,9,55.862463,37.512079,55.864057,37.463808,13.913727,3.027620,321.653
